In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raultruco/cryptowatson-indicators#egg=cryptowatson-indicators
  %pip install --quiet git+https://github.com/mementum/backtrader.git@0fa63ef4a35dc53cc7320813f8b15480c8f85517#egg=backtrader

import backtrader as bt
from cryptowatsonindicators import datas, strategies, RwaIndicator
import requests
requests.packages.urllib3.disable_warnings()
import pprint
pprint = pprint.PrettyPrinter(indent=2, sort_dicts=False, compact=False).pprint   # pprint with defaults

%matplotlib inline

### Global variables

In [ ]:
ticker_symbol = "BTCUSDT"      # currently only works with BTCUSDT
indicator_start_date = None    # Limit indicator series to start at specific date or None to use all the history
binance_api_key = ''           # use your binance keys or leave blank to use Binance's testnet
binance_secret_key = ''
run_get_current = True
run_get_at_date = True
run_plot = True
run_simulator = True

rwa = RwaIndicator(indicator_start_date, ticker_symbol, binance_api_key, binance_secret_key)

### Get rainbow band

In [ ]:
if run_get_current:
    band_index = rwa.get_current_rainbow_band_index()

    rainbow_info = RwaIndicator._get_rainbow_info_by_index(band_index)
    print('Current Rainbow Band:')
    pprint(rainbow_info)

In [ ]:
if run_get_at_date:
    at_date = '01/04/2022'    # date when look up the band
    price_at_date = 30000     # Price of BTC at date
    
    band_index = rwa.get_rainbow_band_index(price=price_at_date, at_date=at_date)

    rainbow_info = RwaIndicator._get_rainbow_info_by_index(band_index)
    print(f"Rainbow Band at {at_date}:")
    pprint(rainbow_info)

### Plot Rainbow chart

In [ ]:
if run_plot:
    rwa.plot_rainbow()

### Simulator

Usage...

In [ ]:

start_date = '01/01/2020'    # start date of the simulation
buy_frequency_days = 5       # Number of days between purchases
buy_amount = 100             # Amount purchased in standard DCA
weight_type = "fibs"         # "fibs" or "originaldca"

if run_simulator:
    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=True)

    cerebro.addstrategy(strategies.RwaStrategy, buy_amount=buy_amount,
                    buy_frequency_days=buy_frequency_days, weight_type=weight_type, log=False, debug=False)

    # Get data feed
    ticker_data = datas.get_nasdaq_ticker_time_series(start_date=start_date)

    ticker_data_feed = bt.feeds.PandasData(
        dataname=ticker_data,
        datetime=0,
        high=None,
        low=None,
        open=1,     # uses the column 1 ('Value') as open price
        close=1,    # uses the column 1 ('Value') as close price
        volume=None,
        openinterest=None,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(ticker_data_feed)

    # Add cash to the virtual broker
    # cerebro.broker.setcash(100000.0)    # default: 10k

    start_portfolio_value = cerebro.broker.getvalue()
    cerebro.run()
    
    end_portfolio_value = cerebro.broker.getvalue()
    pnl_value = end_portfolio_value - start_portfolio_value
    pnl_percent = (pnl_value / start_portfolio_value) * 100
    pnl_sign = '' if pnl_value < 0 else '+'

    # print("----------------------------------------")
    print(f"{'Start value:':<12} {start_portfolio_value:2f} USD")
    print(f"{'Final value:':<12} {end_portfolio_value:2f}  USD")
    print(f"{'PnL:':<11} {pnl_sign}{pnl_value:.2f} USD ({pnl_sign}{pnl_percent:.2f}%)")

    cerebro.plot(volume=False) # volume=False, iplot=True, style='bar' , stdstats=False
